In [1]:
import sys, random, os

from pyevolve import G1DList, GAllele
from pyevolve import GSimpleGA
from pyevolve import Mutators
from pyevolve import Crossovers
from pyevolve import Consts
from pyevolve import DBAdapters
from pyevolve import Selectors
from geopy.distance import geodesic
from math import sqrt

random.seed(1024)

PIL_SUPPORT = None

try:
    from PIL import Image, ImageDraw, ImageFont
    PIL_SUPPORT = True
except ImportError:
    PIL_SUPPORT = False

R = 6371000; # metres
LAT = [16.47, 16.47, 20.09, 22.39, 25.23, 22.00, 20.47, 17.20, 16.30, 14.05, 16.53, 21.52, 19.41, 20.09]
LON = [96.10, 94.44, 92.54, 93.37, 97.24, 96.05, 97.02, 96.29, 97.38, 98.12, 97.38, 95.59, 97.13, 94.55]
CITIES = len(LAT)
WIDTH   = 1024
HEIGHT  = 768
LAST_SCORE = -1

cm     = []
coords = list(zip(LAT, LON))

In [2]:
def cartesian_matrix(coords):
    """ A distance matrix """
    matrix={}
    for i, (x1, y1) in enumerate(coords):
        for j, (x2, y2) in enumerate(coords):
            dx, dy = x1 - x2, y1 - y2
            dist = sqrt(dx * dx + dy * dy)
            matrix[i, j] = dist
    return matrix

In [3]:
def lat_long_cartesian_matrix(coords):
    """ A distance matrix """
    matrix={}
    for i, town1 in enumerate(coords):
        for j, town2 in enumerate(coords):
            # dlat, dlong = math.radians(lat2-lat1), math.radians(long1-long2)
            # latRad1 = math.radians(lat1)
            # latRad2 = math.radians(lat2)
            # a = math.sin(dlat/2) ** 2 + math.cos(latRad1) * math.cos(latRad2) * math.sin(dlong/2) ** 2;
            # c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a));
            # d = R * c;
            matrix[i, j] = geodesic(town1, town2).km
    return matrix

In [4]:
def fitness(matrix, chromosome):
    """ Returns the total length of the tour """
    total = 0
    t = chromosome.getInternalList()
    chromosomeSize = len(t)
    for i in range(chromosomeSize):
        j = (i + 1) % chromosomeSize
        total += matrix[t[i], t[j]]
    return total

In [5]:
def G1DListTSPInitializator(genome, **args):
    """ The initializator for the TSP (We just shuffle the list) """
    lst = [i for i in range(genome.getListSize())]
    random.shuffle(lst)
    genome.setInternalList(lst)

In [6]:
def current_best(ga_engine):
    # Here you have access to the GA Engine
    best = ga.bestIndividual()
    for i in range(0, len(best)):
        print(chr(best[i]))
    return False

In [7]:
global cm, LAT, LON, WIDTH, HEIGHT

cm     = lat_long_cartesian_matrix(coords)
genome = G1DList.G1DList(len(coords))

genome.evaluator.set(lambda chromosome: fitness(cm, chromosome))
genome.crossover.set(Crossovers.G1DListCrossoverEdge)
genome.initializator.set(G1DListTSPInitializator)
genome.mutator.set(Mutators.G1DListMutatorSwap)

# 3662.69
ga = GSimpleGA.GSimpleGA(genome)
ga.setGenerations(500) # default 100
ga.setMinimax(Consts.minimaxType["minimize"])
ga.setCrossoverRate(0.9) #default 0.9
ga.setMutationRate(0.1) # default 0.02
ga.setPopulationSize(80) # default 80
ga.selector.set(Selectors.GTournamentSelector) # default Selectors.GRankSelector
ga.setElitism(True)

# Allows plotting nice graphs by running `python pyevolve_graph.py -0 -m -i run1`
dbadapter = DBAdapters.DBSQLite(identify="run1")
ga.setDBAdapter(dbadapter)

# This is to make a video
ga.evolve(freq_stats=10)

best = ga.bestIndividual()

Gen. 0 (0.00%): Max/Min/Avg Fitness(Raw) [8165.07(8460.03)/5506.63(5225.38)/6804.22(6804.22)]
Gen. 10 (2.00%): Max/Min/Avg Fitness(Raw) [7022.85(7662.03)/4710.90(4087.55)/5852.37(5852.37)]
Gen. 20 (4.00%): Max/Min/Avg Fitness(Raw) [7351.18(7468.09)/4265.12(4087.55)/6125.98(6125.98)]
Gen. 30 (6.00%): Max/Min/Avg Fitness(Raw) [7362.97(7897.91)/4532.95(3834.24)/6135.81(6135.81)]
Gen. 40 (8.00%): Max/Min/Avg Fitness(Raw) [7417.79(7820.40)/4410.87(3834.24)/6181.49(6181.49)]
Gen. 50 (10.00%): Max/Min/Avg Fitness(Raw) [7142.74(8175.94)/4818.37(3834.24)/5952.28(5952.28)]
Gen. 60 (12.00%): Max/Min/Avg Fitness(Raw) [7345.45(7653.86)/4294.44(3834.24)/6121.21(6121.21)]
Gen. 70 (14.00%): Max/Min/Avg Fitness(Raw) [7202.93(7409.79)/4140.38(3819.53)/6002.45(6002.45)]
Gen. 80 (16.00%): Max/Min/Avg Fitness(Raw) [7432.96(7979.21)/4546.18(3819.53)/6194.13(6194.13)]
Gen. 90 (18.00%): Max/Min/Avg Fitness(Raw) [7255.74(7262.26)/3813.50(3801.47)/6046.45(6046.45)]
Gen. 100 (20.00%): Max/Min/Avg Fitness(Raw) [7

In [8]:
print(best)

- GenomeBase
	Score:			 3346.761974
	Fitness:		 3871.150712

	Params:		 {}

	Slot [Evaluator] (Count: 1)
		Name: <lambda> - Weight: 0.50
	Slot [Initializator] (Count: 1)
		Name: G1DListTSPInitializator - Weight: 0.50
		Doc:  The initializator for the TSP (We just shuffle the list) 
	Slot [Mutator] (Count: 1)
		Name: G1DListMutatorSwap - Weight: 0.50
		Doc:  The mutator of G1DList, Swap Mutator

   .. note:: this mutator is :term:`Data Type Independent`

   
	Slot [Crossover] (Count: 1)
		Name: G1DListCrossoverEdge - Weight: 0.50
		Doc:  THe Edge Recombination crossover for G1DList (widely used for TSP problem)

   See more information in the `Edge Recombination Operator <http://en.wikipedia.org/wiki/Edge_recombination_operator>`_
   Wikipedia entry.
   

- G1DList
	List size:	 14
	List:		 [12, 6, 11, 5, 4, 3, 2, 13, 1, 0, 9, 8, 10, 7]


